In [2]:
import jams
import json
import os 
import librosa 
import numpy as np 

In [3]:
audio_path = '../guitar_audio_transcription/data/audio_hex-pickup_debleeded/'
audio_files = os.listdir(audio_path)
anno_path = '../guitar_audio_transcription/data/annotation/'
anno_files = os.listdir(anno_path)

In [4]:
sorted_anno = sorted(anno_files)
sorted_anno[2]

'00_BN1-129-Eb_solo.jams'

In [6]:
'../guitar_audio_transcription/data/annotation'

root = "/Users/michaelbernardrouimi/code/ybouazza-opthread/guitar_audio_transcription/guitar_audio_transcription/data/annotation"

jam_file = os.path.join(root, sorted_anno[2])


In [25]:
## methods for metadata 

def get_start_dur(jams_path: str):
    """
    returns strat time and duration time of the jams file
    
    *requires json and jams modules*
    
    """
    
    r = json.loads(jams.load(jams_path).dumps())
    for element in r['annotations']:
        if element['namespace'] == 'tempo':
            time = element['data'][0]['time']
            duration = element['data'][0]['duration']
            
    return {"time": time, "duration": duration}

def get_timestamps(jam_path):
    
    r = json.loads(jams.load(jam_path).dumps())
    
    o = []

    for element in r['annotations']:

        if element["namespace"]  == "beat_position":
            for timestamp in element["data"]:
                o.append(timestamp["time"])
                
    return o
        

In [11]:
r = json.loads(jams.load(jam_file).dumps())

In [26]:
get_timestamps(jam_file)

[0.0,
 0.46511627906976744,
 0.9302325581395349,
 1.3953488372093024,
 1.8604651162790697,
 2.3255813953488373,
 2.7906976744186047,
 3.255813953488372,
 3.7209302325581395,
 4.186046511627907,
 4.651162790697675,
 5.116279069767442,
 5.5813953488372094,
 6.046511627906977,
 6.511627906976744,
 6.976744186046512,
 7.441860465116279,
 7.906976744186046,
 8.372093023255815,
 8.837209302325581,
 9.30232558139535,
 9.767441860465116,
 10.232558139534884,
 10.69767441860465,
 11.162790697674419,
 11.627906976744185,
 12.093023255813954,
 12.55813953488372,
 13.023255813953488,
 13.488372093023255,
 13.953488372093023,
 14.41860465116279,
 14.883720930232558,
 15.348837209302326,
 15.813953488372093,
 16.27906976744186,
 16.74418604651163,
 17.209302325581394,
 17.674418604651162,
 18.13953488372093,
 18.6046511627907,
 19.069767441860463,
 19.53488372093023,
 20.0,
 20.46511627906977,
 20.930232558139533,
 21.3953488372093,
 21.86046511627907]

In [8]:
get_start_dur(jam_file)

{'time': 0.0, 'duration': 22.32439999999997}

In [9]:
jam = jams.load(jam_file)

In [ ]:

# Initialize variables
cnt_row = -1
cnt_col = 0
cnt_zero = 0

start = 14
stop = 15

# Grab all relevant MIDI data (available in MIDI_dat)
for i in range(0, len(jam['annotations'])):
    if jam['annotations'][i]['namespace'] == 'note_midi':
        for j in range(0, len(sorted(jam['annotations'][i]['data']))):
            cnt_row = cnt_row + 1
            for k in range(0, len(sorted(jam['annotations'][int(i)]['data'])[int(j)]) - 1):
                if cnt_zero == 0:
                    MIDI_arr = np.zeros((len(sorted(jam['annotations'][int(i)]['data'])), len(sorted(jam['annotations'][int(i)]['data'])[int(j)]) - 1), dtype = np.float32)
                    cnt_zero = cnt_zero + 1
                if cnt_zero > 0:
                    MIDI_arr = np.vstack((MIDI_arr, np.zeros((len(sorted(jam['annotations'][int(i)]['data'])), len(sorted(jam['annotations'][int(i)]['data'])[int(j)]) - 1), dtype = np.float32)))
                    cnt_zero = cnt_zero + 1  # Keep
                if cnt_col > 2:
                    cnt_col = 0
                MIDI_arr[cnt_row, cnt_col] = sorted(jam['annotations'][int(i)]['data'])[int(j)][int(k)]
                cnt_col = cnt_col + 1
                
MIDI_dat = np.zeros((cnt_row + 1, cnt_col), dtype = np.float32)
cnt_col2 = 0
for n in range(0, cnt_row + 1):
    for m in range(0, cnt_col):
        if cnt_col2 > 2:
            cnt_col2 = 0
        MIDI_dat[n, cnt_col2] = MIDI_arr[n, cnt_col2]
        cnt_col2 = cnt_col2 + 1
        
# Return the unique MIDI notes played (available in MIDI_val)
MIDI_dat_dur = np.copy(MIDI_dat)
for r in range(0, len(MIDI_dat[:, 0])):
    MIDI_dat_dur[r, 0] = MIDI_dat[r, 0] + MIDI_dat[r, 1]
tab_1, = np.where(np.logical_and(MIDI_dat[:, 0] >= start, MIDI_dat[:, 0] <= stop))
tab_2, = np.where(np.logical_and(MIDI_dat_dur[:, 0] >= start, MIDI_dat_dur[:, 0] <= stop))
tab_3, = np.where(np.logical_and(np.logical_and(MIDI_dat[:, 0] < start, MIDI_dat_dur[:, 0] > stop), MIDI_dat[:, 1] > int(stop-start)))
if tab_1.size != 0 and tab_2.size == 0 and tab_3.size == 0:
    tab_ind = tab_1
if tab_1.size == 0 and tab_2.size != 0 and tab_3.size == 0:
    tab_ind = tab_2
if tab_1.size == 0 and tab_2.size == 0 and tab_3.size != 0:
        tab_ind = tab_3
if tab_1.size != 0 and tab_2.size != 0 and tab_3.size == 0:
    tab_ind = np.concatenate([tab_1, tab_2])
if tab_1.size != 0 and tab_2.size == 0 and tab_3.size != 0:
    tab_ind = np.concatenate([tab_1, tab_3])
if tab_1.size == 0 and tab_2.size != 0 and tab_3.size != 0:
    tab_ind = np.concatenate([tab_2, tab_3])
if tab_1.size != 0 and tab_2.size != 0 and tab_3.size != 0:
    tab_ind = np.concatenate([tab_1, tab_2, tab_3])
if tab_1.size == 0 and tab_2.size == 0 and tab_3.size == 0:
    tab_ind = []
if len(tab_ind) != 0:
    MIDI_val = np.zeros((len(tab_ind), 1), dtype = np.float32)
    for z in range(0, len(tab_ind)):
        MIDI_val[z, 0] = int(round(MIDI_dat[tab_ind[z], 2]))
elif len(tab_ind) == 0:
    MIDI_val = []
MIDI_val = np.unique(MIDI_val)
if MIDI_val.size >= 6:
    MIDI_val = np.delete(MIDI_val, np.s_[6::])
    



In [ ]:
print(MIDI_val)

[58.]


In [ ]:
round(get_start_dur(jam_file)['duration'])

22

In [ ]:
[x / 10.0 for x in range(0, round(get_start_dur(jam_file)['duration']) * 10, 2)]

In [116]:
def get_midi_by_time(jam_file, tempos:list = []):
    # Initialize variables
    cnt_row = -1
    cnt_col = 0
    cnt_zero = 0
    
    if tempos == []:
        times = [x / 10.0 for x in range(0, round(get_start_dur(jam_file)['duration']) * 10, 2)]
        
    else:
        times = tempos
    
    results = []
    results2 = []
    
    jam = jams.load(jam_file)

    for time in times:
        
        start = time
        
        if start == times[-1]:
            print('nothing else')
        
        else:
            
            stop = times[times.index(start) + 1]
            
            # Grab all relevant MIDI data (available in MIDI_dat)
            for i in range(0, len(jam['annotations'])):
                if jam['annotations'][i]['namespace'] == 'note_midi':
                    for j in range(0, len(sorted(jam['annotations'][i]['data']))):
                        cnt_row = cnt_row + 1
                        for k in range(0, len(sorted(jam['annotations'][int(i)]['data'])[int(j)]) - 1):
                            if cnt_zero == 0:
                                MIDI_arr = np.zeros((len(sorted(jam['annotations'][int(i)]['data'])), len(sorted(jam['annotations'][int(i)]['data'])[int(j)]) - 1), dtype = np.float32)
                                cnt_zero = cnt_zero + 1
                            if cnt_zero > 0:
                                MIDI_arr = np.vstack((MIDI_arr, np.zeros((len(sorted(jam['annotations'][int(i)]['data'])), len(sorted(jam['annotations'][int(i)]['data'])[int(j)]) - 1), dtype = np.float32)))
                                cnt_zero = cnt_zero + 1  # Keep
                            if cnt_col > 2:
                                cnt_col = 0
                            MIDI_arr[cnt_row, cnt_col] = sorted(jam['annotations'][int(i)]['data'])[int(j)][int(k)]
                            cnt_col = cnt_col + 1
                            
            MIDI_dat = np.zeros((cnt_row + 1, cnt_col), dtype = np.float32)
            cnt_col2 = 0
            for n in range(0, cnt_row + 1):
                for m in range(0, cnt_col):
                    if cnt_col2 > 2:
                        cnt_col2 = 0
                    MIDI_dat[n, cnt_col2] = MIDI_arr[n, cnt_col2]
                    cnt_col2 = cnt_col2 + 1
                    
            # Return the unique MIDI notes played (available in MIDI_val)
            MIDI_dat_dur = np.copy(MIDI_dat)
            for r in range(0, len(MIDI_dat[:, 0])):
                MIDI_dat_dur[r, 0] = MIDI_dat[r, 0] + MIDI_dat[r, 1]
                
            tab_1, = np.where(np.logical_and(MIDI_dat[:, 0] >= start, MIDI_dat[:, 0] <= stop))
            tab_2, = np.where(np.logical_and(MIDI_dat_dur[:, 0] >= start, MIDI_dat_dur[:, 0] <= stop))
            tab_3, = np.where(np.logical_and(np.logical_and(MIDI_dat[:, 0] < start, MIDI_dat_dur[:, 0] > stop), MIDI_dat[:, 1] > int(stop-start)))
            
            if tab_1.size != 0 and tab_2.size == 0 and tab_3.size == 0:
                tab_ind = tab_1
            
            if tab_1.size == 0 and tab_2.size != 0 and tab_3.size == 0:
                tab_ind = tab_2
            
            if tab_1.size == 0 and tab_2.size == 0 and tab_3.size != 0:
                    tab_ind = tab_3
            
            if tab_1.size != 0 and tab_2.size != 0 and tab_3.size == 0:
                tab_ind = np.concatenate([tab_1, tab_2])
            
            if tab_1.size != 0 and tab_2.size == 0 and tab_3.size != 0:
                tab_ind = np.concatenate([tab_1, tab_3])
            
            if tab_1.size == 0 and tab_2.size != 0 and tab_3.size != 0:
                tab_ind = np.concatenate([tab_2, tab_3])
            
            if tab_1.size != 0 and tab_2.size != 0 and tab_3.size != 0:
                tab_ind = np.concatenate([tab_1, tab_2, tab_3])
                
            if tab_1.size == 0 and tab_2.size == 0 and tab_3.size == 0:
                tab_ind = []
                
            if len(tab_ind) != 0:
                MIDI_val = np.zeros((len(tab_ind), 1), dtype = np.float32)
                for z in range(0, len(tab_ind)):
                    MIDI_val[z, 0] = int(round(MIDI_dat[tab_ind[z], 2]))
            
            elif len(tab_ind) == 0:
                MIDI_val = []
            
            MIDI_val = np.unique(MIDI_val)
            
            if MIDI_val.size >= 6:
                MIDI_val = np.delete(MIDI_val, np.s_[6::])

            results.append(MIDI_val.astype(int).tolist())
    

    
    return results
        


In [117]:
o = get_midi_by_time(jam_file, get_timestamps(jam_file))
o

nothing else


[[46, 51, 53, 56, 58],
 [52, 53, 58],
 [52, 53],
 [52, 53, 57, 58],
 [53, 57, 58],
 [53, 57],
 [53, 57, 62],
 [57, 62],
 [57, 62],
 [57, 58, 62],
 [58, 62, 65],
 [58, 62, 65],
 [58, 62, 65, 70],
 [58, 62, 65, 70],
 [62, 63, 65, 67, 72],
 [62, 65],
 [65, 70, 72],
 [65, 70],
 [55, 69, 70],
 [55],
 [55],
 [55, 67],
 [55, 67],
 [55, 65, 67],
 [55, 63, 65, 67],
 [55, 62, 63],
 [55, 62, 63, 65],
 [55, 62, 63],
 [60, 62],
 [58],
 [58],
 [58],
 [57, 58],
 [57, 58],
 [58, 60],
 [58, 60],
 [60, 62],
 [60, 62],
 [57, 58],
 [57, 58, 60],
 [58, 60],
 [53, 58],
 [53],
 [53, 54],
 [55],
 [55],
 [55]]

[[46, 51, 53, 56, 58],
 [52, 53, 58],
 [52, 53],
 [52, 53, 57, 58],
 [53, 57, 58],
 [53, 57],
 [53, 57, 62],
 [57, 62],
 [57, 62],
 [57, 58, 62],
 [58, 62, 65],
 [58, 62, 65],
 [58, 62, 65, 70],
 [58, 62, 65, 70],
 [62, 63, 65, 67, 72],
 [62, 65],
 [65, 70, 72],
 [65, 70],
 [55, 69, 70],
 [55],
 [55],
 [55, 67],
 [55, 67],
 [55, 65, 67],
 [55, 63, 65, 67],
 [55, 62, 63],
 [55, 62, 63, 65],
 [55, 62, 63],
 [60, 62],
 [58],
 [58],
 [58],
 [57, 58],
 [57, 58],
 [58, 60],
 [58, 60],
 [60, 62],
 [60, 62],
 [57, 58],
 [57, 58, 60],
 [58, 60],
 [53, 58],
 [53],
 [53, 54],
 [55],
 [55],
 [55]]

In [89]:
from operator import itemgetter
 
# printing original list

 
# sort list of list
# sort by second index


res = sorted(i, key = itemgetter(0))

In [92]:

for elem in range(len(res)):
    
    res[elem][2] = round(res[elem][2])
    res[elem] = res[elem]
    
res 

[[0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.18269388377666473, 0.17414966225624084, 56],
 [0.1826938837766647

In [93]:
oioi = np.unique(res, axis=0)

In [94]:
oioi 

array([[ 0.18269388,  0.17414966, 56.        ],
       [ 0.18339683,  0.16834468, 51.        ],
       [ 0.22475737,  0.21478458, 46.        ],
       [ 0.47795466,  0.29024944, 58.        ],
       [ 0.48881632,  0.30185941, 53.        ],
       [ 0.84711564,  0.34829932, 52.        ],
       [ 0.92112923,  0.42376417, 52.        ],
       [ 1.02677548,  0.09868481, 52.        ],
       [ 1.43375957,  0.40634921, 53.        ],
       [ 1.68183231,  0.24380952, 57.        ],
       [ 1.8043946 ,  0.36571428, 52.        ],
       [ 1.89725173,  0.21478458, 58.        ],
       [ 2.75430393,  0.84752834, 57.        ],
       [ 2.90561914,  0.59210885, 53.        ],
       [ 3.71564174,  0.47600907, 62.        ],
       [ 4.19480276,  1.40480721, 57.        ],
       [ 4.20398617,  0.48761904, 62.        ],
       [ 5.0122633 ,  0.55147392, 62.        ],
       [ 5.35183239,  0.43537414, 65.        ],
       [ 6.03192282,  0.20897959, 65.        ],
       [ 6.27210426,  0.4643991 , 70.   

In [ ]:
prueba = np.array([46., 51., 53., 56., 58.])
prueba.tolist(dtype='int')

TypeError: tolist() takes no keyword arguments

In [ ]:
o

[array([46., 51., 53., 56., 58.], dtype=float32),
 array([46., 53., 58.], dtype=float32),
 array([52., 53., 58.], dtype=float32),
 array([52.], dtype=float32),
 array([52.], dtype=float32),
 array([52., 53.], dtype=float32),
 array([53.], dtype=float32),
 array([52., 53., 57.], dtype=float32),
 array([52., 57., 58.], dtype=float32),
 array([52., 57., 58.], dtype=float32),
 array([57.], dtype=float32),
 array([53., 57.], dtype=float32),
 array([53., 57.], dtype=float32),
 array([53., 57.], dtype=float32),
 array([53., 57.], dtype=float32),
 array([57.], dtype=float32),
 array([57., 62.], dtype=float32),
 array([57., 62.], dtype=float32),
 array([57., 62.], dtype=float32),
 array([57., 62.], dtype=float32),
 array([57., 62.], dtype=float32),
 array([58., 62.], dtype=float32),
 array([58., 62.], dtype=float32),
 array([58., 62.], dtype=float32),
 array([58., 62., 65.], dtype=float32),
 array([58., 62., 65.], dtype=float32),
 array([58., 62., 65.], dtype=float32),
 array([58., 62.], dtype=

In [ ]:
times = [x / 10.0 for x in range(0, round(get_start_dur(jam_file)['duration']) * 10, 2)]

for time in times:
    
    start = time
    
    if times[-1] == start:
        print('llegó al final')


llegó al final


In [ ]:
jam.keys()

dict_keys(['annotations', 'file_metadata', 'sandbox'])

In [ ]:
jam['annotations'][-4]

,time,duration,value,confidence
0,0.000,22.324,129.0,1.0


In [ ]:
r = json.loads(jam.dumps())

In [ ]:
r['annotations'][-4]['data'][0]['value']

129.0

In [ ]:
array = []
for element in r['annotations'][-5]['data']:
    array.append(element['time'])
    
array

[0.0,
 0.46511627906976744,
 0.9302325581395349,
 1.3953488372093024,
 1.8604651162790697,
 2.3255813953488373,
 2.7906976744186047,
 3.255813953488372,
 3.7209302325581395,
 4.186046511627907,
 4.651162790697675,
 5.116279069767442,
 5.5813953488372094,
 6.046511627906977,
 6.511627906976744,
 6.976744186046512,
 7.441860465116279,
 7.906976744186046,
 8.372093023255815,
 8.837209302325581,
 9.30232558139535,
 9.767441860465116,
 10.232558139534884,
 10.69767441860465,
 11.162790697674419,
 11.627906976744185,
 12.093023255813954,
 12.55813953488372,
 13.023255813953488,
 13.488372093023255,
 13.953488372093023,
 14.41860465116279,
 14.883720930232558,
 15.348837209302326,
 15.813953488372093,
 16.27906976744186,
 16.74418604651163,
 17.209302325581394,
 17.674418604651162,
 18.13953488372093,
 18.6046511627907,
 19.069767441860463,
 19.53488372093023,
 20.0,
 20.46511627906977,
 20.930232558139533,
 21.3953488372093,
 21.86046511627907]